# Machine Learning: Home Work 1 (II consegna)
<b>Leonardo Serilli</b>
- matricola 274426
- 25/11/2020

In [24]:
import pandas as pd
import numpy as np
import random
import sys 
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
DATA = pd.read_csv('Letters.csv')
features = [i for i in DATA]
alphabets = [DATA[i].unique().tolist() for i in DATA]
print("DATA shape", DATA.shape)
DATA

DATA shape (20000, 17)


lettr  xbox  ybox  width  high  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0         T     2     8      3     5      1     8    13      0      6      6   
1         I     5    12      3     7      2    10     5      5      4     13   
2         D     4    11      6     8      6    10     6      2      6     10   
3         N     7    11      6     6      3     5     9      4      6      4   
4         G     2     1      3     1      1     8     6      6      6      6   
...     ...   ...   ...    ...   ...    ...   ...   ...    ...    ...    ...   
19995     D     2     2      3     3      2     7     7      7      6      6   
19996     C     7    10      8     8      4     4     8      6      9     12   
19997     T     6     9      6     7      5     6    11      3      7     11   
19998     S     2     3      4     2      1     8     7      2      6     10   
19999     A     4     9      6     6      2     9     5      3      1      8   

       x2ybr  xy2br  xege  xegvy  yege  yegvx  
0         10      8     0      8     0      8  
1          3      9     2      8     4     10  
2          3      7     3      7     3      9  
3          4     10     6     10     2      8  
4          5      9     1      7     5     10  
...      ...    ...   ...    ...   ...    ...  
19995      6      4     2      8     3      7  
19996      9     13     2      9     3      7  
19997      9      5     2     12     2      4  
19998      6      8     1      9     5      8  
19999      1      8     2      7     2      8  

[20000 rows x 17 columns]

In [9]:
# approssima i missing values in un data frame con la media sulla feature corrispondente
def nan_subs(df):
    df = pd.DataFrame(df).fillna(df.mean().astype(int))
    return df

In [4]:
# introduce un sample nel data frame al dato indice
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]
    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)
    return df

In [5]:
df = DATA
df_new = copy.deepcopy(df)

# espansione del df_new, introducendo, per ogni sample, k copie, una per ogni feature i, con la feature i mancante
# espansione del df, introducendo k copie di ogni sample, una per ogni feature; al fine di matchare il df_new
old_df = copy.deepcopy(df) #usato poichè il df viene modificato durante il loop
for i, row in old_df.iterrows():
    for j in range(len(features)):
        new_sample = copy.deepcopy(row)
        new_sample[j] = np.nan
        df_new = insert_row(i, df_new, new_sample)
        df = insert_row(i, df, row)   

print("df shape", df.shape)
print("df_new", df_new.shape)

#### esempio di espansione del primo sample nel df_new
<img src="new_df.png">

In [30]:
# approssimazione dei missing value con il valore medio della feature in cui si trova
df_new = nan_subs(df_new)

# split in 80% TS e 20% testSet di entrambi i df
TS, testSet = train_test_split(df, test_size= 0.20, shuffle = False)
TS_new, testSet_new = train_test_split(df_new, test_size= 0.20, shuffle = False)

# Encoding con vettore caratteristico  dei training sets e dei test sets
TS = pd.get_dummies(TS, columns=[name for name in features],prefix=[name for name in features])
TS = TS[:int(0.5*len(TS))]

TS_new = pd.get_dummies(TS_new, columns=[name for name in features],prefix=[name for name in features])
TS_new = TS_new[:int(0.5*len(TS_new))]


testSet = pd.get_dummies(testSet, columns=[name for name in features],prefix=[name for name in features])
testSet = testSet[:int(0.5*len(testSet))]


testSet_new = pd.get_dummies(testSet_new, columns=[name for name in features],prefix=[name for name in features])
testSet_new = testSet_new[:int(0.5*len(testSet_new))]

print("encoded TS shape", TS.shape)
print("encoded TS_new shape", TS_new.shape)
print("ecnoded testSet shape", testSet.shape)
print("ecnoded testSet_new shape", testSet_new.shape)

encoded TS shape (144000, 282)
encoded TS_new shape (144000, 282)
ecnoded testSet shape (36000, 282)
ecnoded testSet_new shape (36000, 282)


### Esempio di encoding in vettore caratteristico
<img src="encTS.png">

In [1]:
# creazione di un modello in grado, da un sample di k feature, di predire un sample di altrettante feature in output
model = MLPClassifier(hidden_layer_sizes=(282,282,282), activation='relu', solver='adam', max_iter=1000)

# training effettuato con un input di k feature, il quale preseta missing values (approssimati), 
 # e un output uguale all' input ma senza missing values
model.fit(TS_new,TS) 

In [ ]:
# predizione del modello su un test set di k feature, con la presenza di missing values (approssimati)
predictions = model.predict(testSet_new)

# accuratezza confrontanto la predizione con il test set senza missing values
acc_scores = accuracy_score(predictions, testSet)
print("Accurancy: ", acc_scores)